In [1]:
from __future__ import print_function
import time, os, json
import tensorflow as tf
import pickle
import os
import numpy as np
import logging
import matplotlib.pyplot as plt

from util import *
from model.video_caption import sequence_2_sequence_LSTM


from cs231n.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from cs231n.image_utils import image_from_url

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


# Build Model

In [2]:
# define parameters
savedModel = True
max_train = 10000
num_frames = 1
batch_size = 512
epochs = 1000
learning_rate = 1e-5
decay_rate = 0.95
decay_steps = 10
word_vector_size = 50
LSTM_output_size = 150
LSTM_hidden_size = 150

In [3]:
data = load_coco_data(pca_features = False, max_train = max_train)
input_size = data['train_features'].shape[1]
maxLen = data['train_captions'].shape[1]
wordLs = []
for caption in data['train_captions']:
    for word in caption:
        wordLs.append(word)
voc_size = len(list(set(wordLs)))

# Print out all the keys and values from the data dictionary
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))
print('\ninput_size:', input_size, ' maxLen:', maxLen, ' voc_size:', voc_size)
print('Finish loading training data!')

val_features <class 'numpy.ndarray'> (40504, 4096) float32
word_to_idx <class 'dict'> 1004
train_urls <class 'numpy.ndarray'> (82783,) <U63
val_urls <class 'numpy.ndarray'> (40504,) <U63
val_image_idxs <class 'numpy.ndarray'> (195954,) int32
train_features <class 'numpy.ndarray'> (82783, 4096) float32
train_captions <class 'numpy.ndarray'> (10000, 17) int32
val_captions <class 'numpy.ndarray'> (195954, 17) int32
idx_to_word <class 'list'> 1004
train_image_idxs <class 'numpy.ndarray'> (10000,) int32

input_size: 4096  maxLen: 17  voc_size: 1004
Finish loading training data!


In [4]:
# create word2ind and ind2word dictionary
index2Word_ori = data['idx_to_word']
word2Index_ori = data['word_to_idx']

wordLs = sorted(list(set(wordLs)))
word2index = {}
index2word = {}
for i, word_ind in enumerate(wordLs):
    word = index2Word_ori[word_ind]
    word2index[word] = i
    index2word[i] = word
print('finish building dictionary')

finish building dictionary


build model

In [5]:
logging.basicConfig(level=logging.INFO)
#=======Change These===============================
tf.app.flags.DEFINE_string("model_name", "baseline_1", "name of the model")
tf.app.flags.DEFINE_integer("state_size", LSTM_hidden_size, "Size of each model layer.")
tf.app.flags.DEFINE_float("learning_rate", learning_rate, "Base Learning rate.")
#==================================================

tf.app.flags.DEFINE_float("input_size", input_size, "input size for each frame")
tf.app.flags.DEFINE_integer("batch_size", batch_size, "how many videos put per run")
tf.app.flags.DEFINE_integer("max_sentence_length", maxLen, "maximum captioning sentence length")
tf.app.flags.DEFINE_integer("word_vector_size", word_vector_size, "word embedding dimension default is 25 for twitter glove")
tf.app.flags.DEFINE_integer("voc_size", voc_size, "vocabulary size")
tf.app.flags.DEFINE_integer("n_epochs", epochs, "number of epoch to run")
tf.app.flags.DEFINE_integer("num_frames", num_frames, "number of frames per video")
tf.app.flags.DEFINE_integer("hidden_size", LSTM_output_size, "output size of LSTM encoder and decoder")
# tf.app.flags.DEFINE_integer("decay_rate", decay_rate, "decay rate of learning")
# tf.app.flags.DEFINE_integer("decay_steps",decay_steps, "how many steps to decay")

FLAGS = tf.app.flags.FLAGS  

In [6]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    return session

is_training = True



captions_train = []
captions_corresponding_videoIds = []
input_frames_train = {}

captions_corresponding_videoIds = data['train_image_idxs']
for i, ind in enumerate(data['train_image_idxs']):
    caption = data['train_captions'][i]
    caption_new = []
    for word_ind in caption:
        word = index2Word_ori[word_ind]
        caption_new.append( word2index[word] )
        
    captions_train.append( (ind, list(caption_new) ) )
    input_frames_train[ind] = data['train_features'][ind].reshape(1, input_size)

word_embedding = np.random.randn(voc_size, word_vector_size).astype(np.float32)

In [7]:
# build model graph
tf.reset_default_graph()
model = sequence_2_sequence_LSTM(word_embedding, FLAGS)
model.train_embedding = True
model.build()

start building model ...


TypeError: object of type 'Tensor' has no len()

In [ ]:
# run training mode
saver = tf.train.Saver()

with get_session() as sess:
    sess.run(tf.global_variables_initializer())
    pack = model.train(sess, (input_frames_train, captions_train), verbose = True)
    save_path = saver.save(sess, "./saved_model/COCO_lstm.ckpt")
    print("Model saved in file: %s" % save_path)
val_loss, tr_loss, tr_pred, val_pred, train_id, val_id = pack

In [ ]:
val_loss, tr_loss, tr_pred, val_pred, train_id, val_id = pack
# plot learning curve
plt.plot(range(len(tr_loss)), tr_loss, 'r-', linewidth = 2, label = 'train')
plt.plot(range(len(val_loss)), val_loss, 'b-', linewidth = 2, label = 'validation')
plt.grid()
plt.xlabel('iteration', fontsize = 13)
plt.ylabel('loss', fontsize = 13)
plt.title('iteration vs loss', fontsize = 15)
plt.legend()
plt.savefig(os.getcwd() + '/output/CoCo_caption_learning_curve_2.png')

In [ ]:
# check captions
batch_size = batch_size
# sample = tr_pred[idx] # training
# # sample = val_pred[idx] # validation 
# words = []
# embed_train = False # whether train embedding word

captions = {k: v for k, v in captions_train}
# caption check
for j in range(batch_size):
    words = []
    trues = []
    sample = tr_pred[j]
    vid = train_id[j]
    cap = captions[vid]
    for idx, i in enumerate(sample):
        word = index2word[i]
        true = index2word[cap[idx]]
        if word not in ['<START>', '<NULL>', '<END>']:
            words.append(word)
        if true not in ['<START>', '<NULL>', '<END>']:
            trues.append(true)   
    print('Pred Caption: ', ' '.join(w for w in words))
    print('True Caption: ', ' '.join(t for t in trues))
    print('-'*50)

# Look At Data

In [ ]:


url = train_urls[vid]
plt.imshow(image_from_url(url))
plt.axis('off')
caption_str = decode_captions(caption, data['idx_to_word'])
plt.title(caption_str)
plt.show()

# # Sample a minibatch and show the images and captions
# batch_size = 1
# captions, features, urls = sample_coco_minibatch(data, batch_size=batch_size)
# for i, (caption, url) in enumerate(zip(captions, urls)):
#     plt.imshow(image_from_url(url))
#     plt.axis('off')
#     caption_str = decode_captions(caption, data['idx_to_word'])
#     plt.title(caption_str)
#     plt.show()